In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import cv2
import tqdm as tqdm
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [2]:
beach = "scene/train/beach"
building = "scene/train/building"
forest = "scene/train/forest"
mountain = "scene/train/mountain"
park = "scene/train/park"

In [3]:
print("Number of Images in Each Directory:")
print(f"beach: {len(os.listdir(beach))}")
print(f"building: {len(os.listdir(building))}")
print(f"forest: {len(os.listdir(forest))}")
print(f"mountain: {len(os.listdir(mountain))}")
print(f"park: {len(os.listdir(park))}")

Number of Images in Each Directory:
beach: 4500
building: 4500
forest: 4500
mountain: 4500
park: 4500


In [4]:
x = []
y = [] 
dataset =[]
def create_dataset(directory,dir_name):
    for i in tqdm.tqdm(os.listdir(directory)):
        full_path = os.path.join(directory,i)
        try:
            img = cv2.imread(full_path)
            img = cv2.resize(img,(128,128))
        except:
            continue
        x.append(img)
        y.append(dir_name)
    return x, y

In [5]:
x,y= create_dataset(beach,"beach")
x,y= create_dataset(building,"building")
x,y= create_dataset(forest,"forest")
x,y= create_dataset(mountain,"mountain")
x,y= create_dataset(park,"park")

100%|██████████| 4500/4500 [00:06<00:00, 704.11it/s]


In [6]:
x =  np.array(x)
y = np.array(y)
x.shape,y.shape

((22500, 128, 128, 3), (22500,))

In [ ]:
fig = plt.figure(figsize=(12,7))
for i in range(15):
    sample =  random.choice(range(len(x)))
    image = x[sample]
    category = y[sample]
    plt.subplot(3,5,i+1)
    plt.subplots_adjust(hspace=0.3)
    plt.imshow(image)
    plt.xlabel(category)
    
plt.tight_layout()
plt.show()

In [7]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

In [8]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [9]:
img_size =128

x_train = np.array(x_train)/255.0
x_test = np.array(x_test)/255.0


x_train = x_train.reshape(-1,img_size,img_size,3)
y_train = np.array(y_train)

x_test = x_test.reshape(-1,img_size,img_size,3)
y_test = np.array(y_test)

In [10]:
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
y_train_lb = lb.fit_transform(y_train)
y_test_lb = lb.fit_transform(y_test)

In [11]:
y_train_lb.shape,y_test_lb.shape

((18000, 5), (4500, 5))

In [13]:
import tensorflow as tf

In [14]:
from tensorflow.keras.applications.vgg19 import VGG19
vgg = VGG19(weights = "imagenet",include_top=False,input_shape=(img_size,img_size,3))

In [15]:
for layer in vgg.layers:
    layer.trainable = False

In [16]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten,Dense
model =Sequential()
model.add(vgg)
model.add(Flatten())
model.add(Dense(5,activation="softmax"))

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Functional)           (None, 4, 4, 512)         20024384  
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 5)                 40965     
Total params: 20,065,349
Trainable params: 40,965
Non-trainable params: 20,024,384
_________________________________________________________________


In [18]:
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics="accuracy")

from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
checkpoint = ModelCheckpoint("vgg19.h5",monitor="val_accuracy",verbose=1,save_best_only=True,
                             save_weights_only=False)
earlystop = EarlyStopping(monitor="val_accuracy",patience=5,verbose=1)

In [19]:
unique,counts = np.unique(y_train_lb,return_counts=True)
print(unique,counts)

[0 1] [72000 18000]


In [20]:
batch_size=32
history =  model.fit(x_train,y_train_lb,epochs=15,validation_data=(x_test,y_test_lb),
                     batch_size=32 ,verbose=1,callbacks=[checkpoint,earlystop])

Epoch 1/15
563/563 [==============================] - ETA: 0s - loss: 0.5426 - accuracy: 0.8124
Epoch 00001: val_accuracy improved from -inf to 0.83822, saving model to vgg19.h5
563/563 [==============================] - 105s 187ms/step - loss: 0.5426 - accuracy: 0.8124 - val_loss: 0.4881 - val_accuracy: 0.8382
Epoch 2/15
563/563 [==============================] - ETA: 0s - loss: 0.4082 - accuracy: 0.8618
Epoch 00002: val_accuracy improved from 0.83822 to 0.83889, saving model to vgg19.h5
563/563 [==============================] - 86s 152ms/step - loss: 0.4082 - accuracy: 0.8618 - val_loss: 0.4733 - val_accuracy: 0.8389
Epoch 3/15
563/563 [==============================] - ETA: 0s - loss: 0.3438 - accuracy: 0.8817
Epoch 00003: val_accuracy improved from 0.83889 to 0.84244, saving model to vgg19.h5
563/563 [==============================] - 86s 153ms/step - loss: 0.3438 - accuracy: 0.8817 - val_loss: 0.4727 - val_accuracy: 0.8424
Epoch 4/15
563/563 [==============================] - ETA

In [21]:
import tensorflow as tf 

if tf.test.gpu_device_name(): 

    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))

else:

   print("Please install GPU version of TF")

Default GPU Device:/device:GPU:0


In [12]:
from tensorflow.keras.models import load_model
model1 = load_model("scene_recog.h5")

In [13]:
from sklearn.metrics import roc_curve
y_pred_keras = model1.predict(x_test).ravel()
fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_test_lb, y_pred_keras)

ValueError: multilabel-indicator format is not supported